# Esther's Text Processing Notebook Using Spacy

In [ ]:
#pip install spacy

In [ ]:
#pip install cupy --use-pep517

In [2]:
import pandas as pd
import spacy

C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [ ]:
#!python -m spacy download en_core_web_sm

In [ ]:
#pip install -U spacy[cuda92]

In [ ]:
#pip install cupy

In [1]:
%run C:\\Users\\esthe\\DSEI2100_Project\\machine-learning-dse-i210-final-project-fake-news\\data\\external\\ML_dataset_dropbox.py

In [1]:
df = df.dropna()

NameError: name 'df' is not defined

In [ ]:
# remove extra column
df = df.drop('Unnamed: 0', axis = 1)

In [ ]:
spacy.require_gpu()

In [ ]:
# load the english model
nlp = spacy.load('en_core_web_sm')

In [ ]:
# process the text (takes much longer to run than nltk)
# df['lem_toks_spcy'] = df['Tokens'].apply(lambda tokens: [nlp(token) for token in tokens])